In [6]:
import requests
import base64
import json

In [7]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

# OAuth Authorization

In [8]:
import flask
import urllib.parse as urlparse

code = None
app = flask.Flask(__name__)

scopes = " ".join([
    "user-read-playback-state"
    ,"user-read-currently-playing"
    ,"playlist-read-private"
    ,"playlist-read-collaborative"
    ,"user-read-playback-position"
    ,"user-top-read"
    ,"user-read-recently-played"
    ,"user-library-read"
    ,"user-read-email"
    ,"user-read-private"])

@app.route("/")
def index():
    
    user_auth_url = 'https://accounts.spotify.com/authorize?' + urlparse.urlencode({
        'client_id': client_id,
        'response_type': 'code',
        'scope': scopes,
        'redirect_uri': redirect_url,
    })  
    return flask.redirect(user_auth_url)

@app.route("/callback")
def callback():
    code = flask.request.args.get('code')
    if code is None:
        err = flask.request.args.get('error')
        shutdown_server()
        return f"Authorization failed with the following error: {err}"
    else:
        with open('auth_code.txt','w+') as file:
            file.write(code)

        shutdown_server()
        return "<h1> Authorization successful. Please close this window. </h1>"

def shutdown_server():
    func = flask.request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()

if __name__ == "__main__":
    app.run(host="localhost", port=5050)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jul/2024 22:45:20] "GET / HTTP/1.1" 302 -
/var/folders/_m/s8jbyc4n0yjb3n2vnvp2pc4h0000gn/T/ipykernel_56783/3966248663.py:48: UserWarning: The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.
  func()
127.0.0.1 - - [26/Jul/2024 22:45:20] "GET /callback?code=AQAS3uMyGfJFeVO_qQQlGVvLVKJC3wgWL0pwUXNB7JcW9iIVUAQtUaeOFufqIrA7UjER6Zjp6V7NMVxndkcdFw66W--fSaCNArom0lPiusKoWq8dn0L8jDQghE6u_hvKoE95k-gbUOKxpvFasozy2PrBZY9iJ2Fm1ap8x1ifKKxfBghFeUeZAI0CQJtbiJuAzdpt6l5KR5QwaUCUiZzOyJHa64T5vYb7g3Y52SiZEGEON7VrRvYs4Tt_8A-KL4cNOgVxNId0DMhEI2okQ-wnMRdoIGUDAPsIeVl7USTnCmN1Sn0_YiYrFRL7CY9Wki-A2Rv1c62mJQcA6jhAmhsWphTNTIQTxp-QIVeE1ngR2JvMIGwUQRoaXsypSZ8o3VKFrNxqsjTroq2rmAxN_wB05gOa3TqsLes8V2UTNLqSoMWTO2wSG_zl0ZDk5-885WpykAChZgbQmRTgkD2SrPwZr5f6se1Qz00P8JyG HTTP/1.1" 200 -


# Get Token

In [9]:
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_encoded = str(base64.b64encode(auth_bytes), "utf-8")
    
    auth_api_url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_encoded,
        "Content-type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = requests.post(auth_api_url, headers=headers, data=data)
    json_results = json.loads(result.content)
    token = json_results["access_token"]
    return(token)

In [10]:
def get_auth_tokens():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_encoded = str(base64.b64encode(auth_bytes), "utf-8")
    
    with open('auth_code.txt','r+') as file:
        auth_code = file.read()
    
    token_endpoint = "https://accounts.spotify.com/api/token"
    data = {
        "grant_type":"authorization_code",
        "code":auth_code,
        "redirect_uri":redirect_url,
    }
    headers = {
        "Authorization": "Basic " + auth_encoded,
        "Content-type": "application/x-www-form-urlencoded"
    }
    result = requests.post(token_endpoint, headers=headers, data=data)
    json_results = json.loads(result.content)
    return json_results

In [11]:
output = get_auth_tokens()
if 'error' in output.keys():
    print("Error! ", output["error_description"])
else:
    with open('auth_tokens.json','w+') as file:
        json.dump(output, file, indent=4)

In [12]:
def refresh_access_tokens(refresh_token):
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_encoded = str(base64.b64encode(auth_bytes), "utf-8")
    
    refresh_endpoint = 'https://accounts.spotify.com/api/token'
    data = {
        "grant_type":"refresh_token",
        "refresh_token":refresh_token
    }
    headers = {
        "Authorization": "Basic " + auth_encoded,
        "Content-type": "application/x-www-form-urlencoded"
    }
    
    result = requests.post(refresh_endpoint, headers=headers, data=data)
    json_results = json.loads(result.content)
    return json_results

In [13]:
with open('auth_tokens.json','r+') as file:
    auth_keys = json.load(file)
    
if 'refresh_token' not in auth_keys.keys():
    with open('refresh_token.json','r+') as file:
        refresh_keys = json.load(file)
    refresh_token = refresh_keys['refresh_token']
else:
    refresh_token = auth_keys['refresh_token']
tokens = refresh_access_tokens(refresh_token)

In [14]:
if 'refresh_token' not in tokens.keys():
    with open('refresh_token.json','w+') as file:
        file.write(json.dumps({"refresh_token":f"{refresh_token}"}))

with open('auth_tokens.json','w+') as file:
        json.dump(tokens, file, indent=4)

# Get User Details

In [15]:
with open('auth_tokens.json','r+') as file:
    auth_keys = json.load(file)
    
token = auth_keys["access_token"]

result = requests.get("https://api.spotify.com/v1/me", headers={"Authorization": "Bearer " + token,
        "Content-type": "application/x-www-form-urlencoded"})
json_results = json.loads(result.content)

print(json_results)

{'display_name': 'sdspot2034', 'external_urls': {'spotify': 'https://open.spotify.com/user/sdspot2034'}, 'href': 'https://api.spotify.com/v1/users/sdspot2034', 'id': 'sdspot2034', 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b8280153af8b79b3144b24f1291', 'height': 64, 'width': 64}, {'url': 'https://i.scdn.co/image/ab6775700000ee8580153af8b79b3144b24f1291', 'height': 300, 'width': 300}], 'type': 'user', 'uri': 'spotify:user:sdspot2034', 'followers': {'href': None, 'total': 9}, 'country': 'IN', 'product': 'premium', 'explicit_content': {'filter_enabled': False, 'filter_locked': False}, 'email': 'shreyandas2034@gmail.com'}


In [16]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
# print(yesterday.timestamp()*1e3)
cdc_time = yesterday.timestamp()*1e3

# Get All Songs Played in the last one day

In [18]:
with open('auth_tokens.json','r+') as file:
    auth_keys = json.load(file)
    
token = auth_keys["access_token"]

result = requests.get("https://api.spotify.com/v1/me/player/recently-played", 
        headers={"Authorization": "Bearer " + token, "after": str(cdc_time)})


In [19]:
json_results = json.loads(result.content)
print(json_results)

{'items': [{'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6uFYqDeLIwfURuF1yxVqJU'}, 'href': 'https://api.spotify.com/v1/artists/6uFYqDeLIwfURuF1yxVqJU', 'id': '6uFYqDeLIwfURuF1yxVqJU', 'name': 'J. Maya', 'type': 'artist', 'uri': 'spotify:artist:6uFYqDeLIwfURuF1yxVqJU'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW

In [20]:
import pprint
pprint.pprint(json_results)

{'cursors': {'after': '1722009100598', 'before': '1721981805039'},
 'href': 'https://api.spotify.com/v1/me/player/recently-played',
 'items': [{'context': None,
            'played_at': '2024-07-26T15:51:40.598Z',
            'track': {'album': {'album_type': 'single',
                                'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6uFYqDeLIwfURuF1yxVqJU'},
                                             'href': 'https://api.spotify.com/v1/artists/6uFYqDeLIwfURuF1yxVqJU',
                                             'id': '6uFYqDeLIwfURuF1yxVqJU',
                                             'name': 'J. Maya',
                                             'type': 'artist',
                                             'uri': 'spotify:artist:6uFYqDeLIwfURuF1yxVqJU'}],
                                'available_markets': ['AR',
                                                      'AU',
                                                      'AT',
    